In [50]:
using CSV, DataFrames, GLM, Statistics, Plots
include("data_utils.jl")

o_mse_calc (generic function with 1 method)

#### Data Description
Q1 is income, Q2 is age, Q3 is education, and Q4 is car ownership. 

### 1. Generate Baseline OLS Coefficients

In [2]:
datapath = "data/raw/baseline_b.csv"
coef_base, data = run_ols(datapath)
""

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Q1 ~ 1 + Q2 + Q3 + Q4

Coefficients:
───────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error    t value  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)  -31.4325     26.2397    -1.1979      0.2338  -83.4849    20.6199
Q2             0.248228    0.262209   0.946679    0.3461   -0.271925   0.768381
Q3             3.69354     1.55145    2.3807      0.0192    0.615874   6.7712
Q4            17.9271      8.04848    2.22739     0.0281    1.96105   33.8931
───────────────────────────────────────────────────────────────────────────────


""

### II. Calculate Gradient in First Step 

In [6]:
datapath="data/raw/e_step1_all.csv"
beta_1 = run_gradient(datapath, coef_base)

6×11 DataFrame
│ Row │ Duration │ Q1    │ Q2    │ Q3    │ Q4    │ SC0   │ bonus  │ perturb1 │ perturb2 │ perturb3 │ pg1score │
│     │ Int64    │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64? │ Int64    │ Int64    │ Int64    │ Int64?   │
├─────┼──────────┼───────┼───────┼───────┼───────┼───────┼────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 24       │ 3     │ 2     │ 3     │ 2     │ 1     │ 10     │ 2        │ 1        │ 1        │ 29       │
│ 2   │ 20       │ 5     │ 2     │ 6     │ 1     │ 1     │ 85     │ 1        │ 2        │ 2        │ 29       │
│ 3   │ 22       │ 1     │ 5     │ 6     │ 1     │ 1     │ 90     │ 1        │ 2        │ 2        │ 59       │
│ 4   │ 23       │ 3     │ 3     │ 5     │ 1     │ 1     │ 77     │ 1        │ 2        │ 2        │ 39       │
│ 5   │ 18       │ 8     │ 2     │ 5     │ 1     │ 1     │ 76     │ 1        │ 2        │ 2        │ 29       │
│ 6   │ 18       │ 3     │ 2     │ 5     │ 1     │ 1     │ 80     │ 2        │ 2        │

4-element Array{Float64,1}:
 -30.968405840316372
   0.26068072523396274
   3.979336160868521
   9.001502337522767

### III. Calculate Out of Sample MSE

In [13]:
blist = ["data/raw/baseline_c.csv", "data/raw/baseline_d.csv"] 
e1list = ["data/raw/e_step1_c.csv", "data/raw/e_step1_d.csv"] 
e2list = ["data/raw/e_step2_all.csv"]

1-element Array{String,1}:
 "data/raw/e_step2_all.csv"

In [19]:
mse_base = o_mse_calc(blist, coef_base) 
mse_1 = o_mse_calc(e1list, coef_base) 
mse_2 = o_mse_calc(e2list, beta_1) 

[673.6045749082185, 869.1687944988731]
[1034.1328586545324, 922.9881842386307]
[831.6845743725677]


831.6845743725677

In [43]:
plot(["Baseline", "Wave 1", "Wave 2"], [ mse_base, mse_1, mse_2], seriestype=:bar, ylim =[500, 1150],label=nothing, ylabel="Out of Sample MSE")
savefig("figures/omse.pdf")

### IV. Calculate Summary Statistics

In [51]:
summarize_data("data/raw/baseline_all.csv")
summarize_data("data/raw/e_step1_all.csv")
summarize_data("data/raw/e_step2_all.csv")

4×3 DataFrame
│ Row │ variable │ mean     │ std      │
│     │ Symbol   │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┤
│ 1   │ Q1       │ 48.5506  │ 29.3146  │
│ 2   │ Q2       │ 35.7977  │ 10.0986  │
│ 3   │ Q3       │ 15.8113  │ 1.77194  │
│ 4   │ Q4       │ 0.898833 │ 0.301844 │
514
4×3 DataFrame
│ Row │ variable │ mean    │ std      │
│     │ Symbol   │ Float64 │ Float64  │
├─────┼──────────┼─────────┼──────────┤
│ 1   │ Q1       │ 50.0589 │ 31.8769  │
│ 2   │ Q2       │ 37.1572 │ 11.2274  │
│ 3   │ Q3       │ 15.9725 │ 2.09029  │
│ 4   │ Q4       │ 0.86444 │ 0.342657 │
509
4×3 DataFrame
│ Row │ variable │ mean     │ std      │
│     │ Symbol   │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┤
│ 1   │ Q1       │ 50.5296  │ 29.1743  │
│ 2   │ Q2       │ 37.7833  │ 11.3367  │
│ 3   │ Q3       │ 15.8818  │ 2.08955  │
│ 4   │ Q4       │ 0.894089 │ 0.308104 │
406


### 2. Set Reward Function

In [25]:
dscale = 3.0/3.0
beta = coef(olsb)
println(beta)
perturb = [0, beta[2:4]...]./3
println(mean(data.Q1))
xmeans = [1, mean(data.Q2), mean(data.Q3), mean(data.Q4)]
println("means: ", xmeans)
println("perturbs: ", perturb)
println("mean prediction: ", sum(beta.*xmeans)*dscale)
println("mean prediction at max perturb: ", sum((beta.+perturb).*xmeans)*dscale)
println("mean prediction at min perturb: ", sum((beta.-perturb).*xmeans)*dscale)
println("maximum possible prediction: ", sum((beta.+ perturb).*[1, 85, 20, 1])*dscale)
println("minimum possible prediction: ", sum((beta.-perturb).*[1, 21, 10, 0])*dscale)
println("minimum no perturb: ", sum((beta).*[1, 21, 10, 0])*dscale)
println("maximum no perturb: ", sum(beta.*[1, 85, 20, 1])*dscale)



[-33.60660581428372, 0.05250259161189099, 4.101686529370897, 17.16091185862718]
48.55058365758755
means: [1.0, 35.797665369649806, 15.811284046692608, 0.8988326848249028]
perturbs: [0.0, 0.01750086387063033, 1.3672288431236324, 5.720303952875727]
mean prediction: 48.55058365758755
mean prediction at max perturb: 75.93631348154464
mean prediction at min perturb: 21.164853833630463
maximum possible prediction: 104.60321116312409
minimum possible prediction: -5.526992669244592
minimum no perturb: 8.512813903274967
maximum no perturb: 70.05075691877215


## Naive Step 1

In [4]:
xmeans = [1, mean(data.Q2), mean(data.Q3), mean(data.Q4)]
println(mean(data.Q1))
println(xmeans)

49.851485148514854
[1.0, 37.91089108910891, 15.613861386138614, 0.900990099009901]


In [3]:
datapath = "data/raw/n_step1b.csv"
data = DataFrame!(CSV.File(datapath))
recode_raw_survey_data!(data)
ols = lm(@formula(Q1 ~ Q2 +Q3 +Q4), data)

#yhat = predict(olsb, data)
#mean((yhat .- data[!,:Q1]).^2)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Q1 ~ 1 + Q2 + Q3 + Q4

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                Estimate  Std. Error    t value  Pr(>|t|)    Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)  -53.5058      28.9929    -1.84548     0.0680  -111.049      4.03709
Q2            -0.0953405    0.259475  -0.367437    0.7141    -0.610326   0.419645
Q3             6.06577      1.75501    3.45626     0.0008     2.58256    9.54898
Q4            13.609       10.1241     1.34421     0.1820    -6.48459   33.7026
─────────────────────────────────────────────────────────────────────────────────

In [30]:
data.Q2

303-element SentinelArrays.SentinelArray{Int64,1,Int64,Missing,Array{Int64,1}}:
 29
 29
 69
 29
 49
 39
 39
 49
 29
 21
 39
 21
 29
  ⋮
 39
 39
 29
 69
 21
 29
 39
 21
 29
 49
 49
 29

In [4]:
xmeans = [1, mean(data.Q2), mean(data.Q3), mean(data.Q4)]
println(mean(data.Q1))

println("stds: ", [ std(data.Q1), std(data.Q2), std(data.Q3), std(data.Q4)])
println("means: ", xmeans)


49.851485148514854
stds: [31.791944306259047, 11.53264844682347, 1.7318793088994564, 0.3001649711425204]
means: [1.0, 37.91089108910891, 15.613861386138614, 0.900990099009901]


In [39]:
datapath = "data/raw/e_step1_all.csv"
data = DataFrame!(CSV.File(datapath))
recode_raw_survey_data!(data)
print(size(data))
ols = lm(@formula(Q1 ~ Q2 +Q3 +Q4), data)
ebase = coef(ols)


thread = 1 warning: only found 16 / 17 columns around data row: 510. Filling remaining columns with `missing`
(509, 17)

4-element Array{Float64,1}:
 -15.78402971041747
   0.004511170223513669
   3.217530025497887
  16.523255120153074

In [28]:
(4.1-3.2)/(sqrt(0.66^2+0.7^2))

0.9354720936668208

In [30]:
println((0.045)/(sqrt(0.12^2*2)))

println(0.7/sqrt(4.03^2*2))

0.2651650429449553
0.12282251782396604


In [201]:
println(mean(data.Q1))
xmeans = [1, mean(data.Q2), mean(data.Q3), mean(data.Q4)]
println("means: ", xmeans)


50.05893909626719
means: [1.0, 37.15717092337918, 15.972495088408644, 0.8644400785854617]


### 3. Update Coefficients Based on Experimental Gradient Estimate

In [200]:
coef0 = copy(cbase)
println(coef0)
perturb = coef0[2:length(coef0)]./3
datapath = "data/raw/e_step1_all.csv"
data = DataFrame!(CSV.File(datapath))
recode_raw_survey_data!(data)
println(var(data.Q1))
recode_perturb!(data, perturb)
data_in = ExperimentData(data.Q1, Matrix(data[!, ["Q2", "Q3", "Q4"]]), Matrix(data[!, ["perturb1", "perturb2", "perturb3"]]))
b_new = update_β_robust(data_in, coef0)
println(b_new)
println(b_new/3)
check_bonus_payments(data, b_new)

mean(data.Q1)

[-31.432508727797035, 0.24822801746696074, 3.6935366284379993, 17.927060497589046]
1016.1382516281734
[4.385047638240876, -996.216621360158, -42.86992986457825, 26.77667448019884]
[-30.968405840316372, 0.26068072523396274, 3.979336160868521, 9.001502337522767]
[-10.322801946772124, 0.08689357507798758, 1.3264453869561736, 3.000500779174256]
perturbs: [0.0, 0.08689357507798758, 1.3264453869561736, 3.000500779174256]
mean prediction: 50.058939096267196
mean prediction at max perturb: 77.06805407512836
mean prediction at min perturb: 23.049824117406004
maximum possible prediction: 116.69304375939032
minimum possible prediction: -0.7899679479174182
minimum no perturb: 14.299250998282055
maximum no perturb: 79.77768135946364


50.05893909626719

In [204]:
yhat = coef0[1] .+ sum((coef0[2:(3+1)]').*data_in.x, dims=2)
-2.0/length(data_in.y).*(data_in.y - yhat)'*data_in.x

1×3 Array{Float64,2}:
 -13.3331  9.82014  -1.94325

In [205]:
coef0 = copy(b_new)
println(coef0)
perturb = coef0[2:length(coef0)]./3
datapath = "data/raw/e_step2_all.csv"
data = DataFrame!(CSV.File(datapath))
recode_raw_survey_data!(data)
recode_perturb!(data, perturb)
data_in = ExperimentData(data.Q1, Matrix(data[!, ["Q2", "Q3", "Q4"]]), Matrix(data[!, ["perturb1", "perturb2", "perturb3"]]))
b_new = update_β_robust(data_in, coef0)
println(b_new)
println(b_new/3)

check_bonus_payments(data, b_new)

mean(data.Q3)

[-24.88061827809439, 0.255493940072227, 4.4132139018119005, -4.8461412084979845]
[1.185965975286026, 435.1531485534837, -60.163869602605445, -75.65523508201478]
[-53.59265531423395, 0.25005452571530845, 4.81430636582927, 20.372270485506945]
[-17.864218438077984, 0.08335150857176948, 1.6047687886097568, 6.790756828502315]
perturbs: [0.0, 0.08335150857176948, 1.6047687886097568, 6.790756828502315]
mean prediction: 50.529556650246306
mean prediction at max perturb: 85.2369606384064
mean prediction at min perturb: 15.822152662086225
maximum possible prediction: 130.29138800295746
minimum possible prediction: -17.9965161820245
minimum no perturb: -0.19844661591977086
maximum no perturb: 84.32037717365962


15.881773399014778

In [192]:
run_ols("data/raw/e_step2_all.csv")

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Q1 ~ 1 + Q2 + Q3 + Q4

Coefficients:
──────────────────────────────────────────────────────────────────────────────
              Estimate  Std. Error   t value  Pr(>|t|)    Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   9.22111    11.5873    0.795796    0.4266  -13.5581     32.0003
Q2            0.194891    0.124266  1.56834     0.1176   -0.0494012   0.439184
Q3            0.964005    0.694416  1.38822     0.1658   -0.401135    2.32914
Q4           20.8421      4.72018   4.41554     <1e-4    11.5628     30.1214
──────────────────────────────────────────────────────────────────────────────


([9.221108679274403, 0.19489149181243456, 0.9640046169041485, 20.842128435299987], 406×9 DataFrame. Omitted printing of 3 columns
│ Row │ Duration (in seconds) │ Q1    │ Q2    │ Q3    │ Q4    │ bonus  │
│     │ Int64                 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64? │
├─────┼───────────────────────┼───────┼───────┼───────┼───────┼────────┤
│ 1   │ 15                    │ 25    │ 29    │ 16    │ 1     │ 76     │
│ 2   │ 30                    │ 15    │ 39    │ 16    │ 0     │ 18     │
│ 3   │ 11                    │ 45    │ 29    │ 20    │ 1     │ 97     │
│ 4   │ 21                    │ 45    │ 21    │ 16    │ 1     │ 30     │
│ 5   │ 24                    │ 35    │ 29    │ 16    │ 0     │ 64     │
│ 6   │ 35                    │ 55    │ 29    │ 18    │ 0     │ 21     │
│ 7   │ 32                    │ 75    │ 29    │ 16    │ 1     │ 28     │
│ 8   │ 25                    │ 55    │ 29    │ 18    │ 1     │ 81     │
│ 9   │ 26                    │ 15    │ 29    │ 12    │ 0     │ 37 